In [ ]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate, Activation, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2

import numpy as np
import random
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import pickle
import h5py 
from math import sqrt, log1p ,exp
import math
import re


from tensorflow.keras import Model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import os
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import xlrd
import xlsxwriter
import matplotlib.pyplot as plt

In [ ]:
def openfile():
    root = tk.Tk()
    root.withdraw()
    default_dir = r"文件路径"
    fname = filedialog.askopenfilename(title=u'请选择输入数据', initialdir=(os.path.expanduser(default_dir)))  # 获得选择好的文件
    print('您正在处理数据：', fname)
    return fname
def save():
    root = tk.Tk()
    root.withdraw()
    default_dir = r"文件路径"
    fname = filedialog.asksaveasfilename(title=u'请选择文件保存路径和文件名', initialdir=(os.path.expanduser(default_dir)))
    return fname
def excel2(path,sheet):
    pd_data=pd.read_excel(fname,sheet_name=sheet)
    return np.array(pd_data)


from scipy import stats
np.seterr(divide='ignore', invalid='ignore') #忽略0为除数的警告

def uncertainty(y_true, y_pred):
    median_log_diff = np.median(np.abs(np.log(y_true/y_pred)))
    uncertainty = 100 * (exp(median_log_diff) - 1)
    return uncertainty


def rmsld(y_true, y_pred):
    y_true = np.reshape(y_true,(1,-1))
    y_pred = np.reshape(y_pred,(1,-1))

    log_diff = np.log10(np.abs(y_true/y_pred))
    squared_log_diff = np.square(log_diff)
    mean_squared_log_diff = np.mean(squared_log_diff)
    rmsld = sqrt(mean_squared_log_diff)
    return rmsld

def Rr(y_true, y_pred):
    y_true = np.reshape(y_true,(1,-1))
    y_pred = np.reshape(y_pred,(1,-1))
    y1 = y_true - np.nanmean(y_true)
    y2 = y_pred - np.nanmean(y_pred)
    r1 = (np.nansum(y1 * y2)) ** 2
    r2 = np.nansum(y1 ** 2) * np.nansum(y2 ** 2)
    Rr = r1 / r2
    return Rr

#误差评价
def mae_mean(y,y_pred):
    y = y.reshape(1,-1)
    ye = y_pred.reshape(1,-1)
    mae = np.mean(np.abs(y-ye))
    return mae
def mape(y,y_pred):
    y = y.reshape(1,-1)
    ye = y_pred.reshape(1,-1)
    y1 = (np.abs(y-ye))/y
    mape = np.mean(y1)
    return mape*100
def rmse(y,y_pred):
    n = len(y)
    y = np.reshape(y,(-1,1))
    y_pred = np.reshape(y_pred,(-1,1))
    c = (y-y_pred)**2
    return np.sqrt(np.sum(c)/n)
def mean_relative_error(y,y_pred):
    y = np.reshape(y,(1,-1))
    y_pred = np.reshape(y_pred,(1,-1))    
    mean_relate_error = np.mean(np.abs(y_pred-y)/y)
    return mean_relate_error*100

def MdLQ(y,y_estimate):
    yy = np.log10(np.abs(y_estimate/y))
    return np.median(yy)
def Bias(y,y_estimate):
    m = MdLQ(y,y_estimate)
    yr = np.exp(np.abs(m))-1
    yr = np.sign(m)*yr*100  #np.sign() 取正负号
    return yr

def Bias2(y,y_estimate):
    m = MdLQ(y,y_estimate)
    yr = math.pow(10,np.abs(m))
    yr = np.sign(m)*yr*100  
    return yr

def Error(y,y_estimate):
    y = y
    ye = y_estimate
    yr = np.log10(np.abs(ye/y))
    yr = np.median(np.abs(yr))
    yr = (np.exp(yr)-1)*100
    return yr
def Slope(y,y_estimate):
    y = y.reshape(1,-1)
    ye = y_estimate.reshape(1,-1)
    a1,a2,a3,a4,a5 = stats.linregress(ye, y)
    return a1


def r2metrics(y,y_pred,model_metrics_name = [ Rr,rmse,mae_mean,rmsld,Error,Bias,Slope,Bias2]):
    model_metrics_list = []
    tmp_list = [] # 每个内循环的临时结果列表
    for m in model_metrics_name:  # 循环每个指标对象
        tmp_score = m(y, y_pred)  # 计算每个回归指标结果
        tmp_score = np.round(tmp_score, 3)#将计算结果保留3位小数
        tmp_list.append(tmp_score)  # 将结果存入每个内循环的临时结果列表
    model_metrics_list.append(tmp_list)  # 将结果存入回归评估指标列表
    return model_metrics_list

In [ ]:
model_metrics_name = [Rr,rmse,mae_mean,rmsld,Error,Bias,Slope,Bias2]

# 模型定义

In [ ]:
imputations = 10
epsilon = tf.constant(1e-3)

y_dim = 3  # 输出三个参数
x_dim = 11  # 输入层的维度

N_MIXES =  15 # number of mixture gaussion   
n_mix   = N_MIXES

OUTPUT_DIMS = y_dim  # output dimension
n_targets   = y_dim

In [ ]:
def _calculate_top(prior, values):
    vals, idxs  = tf.nn.top_k(prior, k=1)
    idxs = tf.stack([tf.range(tf.shape(idxs)[0]), tf.reshape(idxs, [-1])], axis=-1)
    return tf.gather_nd(values, idxs)

def _get_top_estimate( coefs):
    prior, mu, _ = coefs
    return _calculate_top(prior, mu)

def _get_top_estimate2( p_test ,n_mixes , output_dims):
    
    n_mixes = n_mixes
    output_dims = output_dims
    n_sig = int(-N_MIXES*OUTPUT_DIMS*OUTPUT_DIMS/2)

    pis  =  np.apply_along_axis((lambda a: a[:n_mixes]),1, p_test)
    mus  =  np.apply_along_axis((lambda a: a[n_mixes:n_mixes+n_mixes*output_dims]),1, p_test)
    sigs = np.apply_along_axis((lambda a: a[n_sig:]),1, p_test)
    
    return _calculate_top(pis, mus)


def get_coefs( output):
    prior, mu, scale = _parse_outputs(output)
    return prior, mu, _covariance(scale)
def _covariance(scale):
    return tf.einsum('abij,abjk->abik', tf.transpose(scale, perm=[0,1,3,2]), scale)

def softmax(w, t=1.0):
    e = np.array(w) / t  
    e -= e.max()  
    e = np.exp(e)
    dist = e / np.sum(e)
    return dist

def elu_plus_one_plus_epsilon(x):
    return (K.elu(x) + 1 + 1e-8)

def loss_func(y_true, y_pred):
    num_mixes = N_MIXES
    output_dim = OUTPUT_DIMS
    y_pred = tf.reshape(y_pred, [-1, (2 * num_mixes * output_dim) + num_mixes], name='reshape_ypreds')
    y_true = tf.reshape(y_true, [-1, output_dim], name='reshape_ytrue')
    out_mu, out_sigma, out_pi = tf.split(y_pred, num_or_size_splits=[num_mixes * output_dim,
                                                                     num_mixes * output_dim,
                                                                     num_mixes],
                                         axis=-1, name='mdn_coef_split')
    # Construct the mixture models
    cat = tfd.Categorical(logits=out_pi)
    component_splits = [output_dim] * num_mixes
    mus = tf.split(out_mu, num_or_size_splits=component_splits, axis=1)
    sigs = tf.split(out_sigma, num_or_size_splits=component_splits, axis=1)
    coll = [tfd.MultivariateNormalDiag(loc=loc, scale_diag=scale) for loc, scale
            in zip(mus, sigs)]
    mixture = tfd.Mixture(cat=cat, components=coll)
    loss = mixture.log_prob(y_true)
    loss = tf.negative(loss)
    loss = tf.reduce_mean(loss)
    return loss

def _parse_outputs(output):
    prior, mu, scale = tf.split(output, [n_mix, n_mix * n_targets, -1], axis=1)
    prior = tf.reshape(prior, shape=[-1, n_mix])
    mu    = tf.reshape(mu,    shape=[-1, n_mix, n_targets])
    scale = tf.reshape(scale, shape=[-1, n_mix, n_targets, n_targets])
    return prior, mu, scale

distribution = 'MultivariateNormalTriL'
def loss_func2( y, output):
    prior, mu, scale = _parse_outputs(output) 
    dist  = getattr(tfp.distributions, distribution)(mu, scale)
    prob  = tfp.distributions.Categorical(probs=prior)
    mix   = tfp.distributions.MixtureSameFamily(prob, dist)

    def impute(mix, y, N):
        return tf.reduce_mean([
            mix.log_prob( tf.where(tf.math.is_nan(y), mix.sample(), y) )
        for _ in range(N)], 0)


    likelihood = tf.cond(tf.reduce_any(tf.math.is_nan(y)), lambda: impute(mix, y, imputations), lambda: mix.log_prob(y))
    return tf.reduce_mean(-likelihood) + tf.add_n([0.] + model.losses)



def sample_from_output(params, output_dim, num_mixes, num_sample, temp=1.0, sigma_temp=1.0):
    mus, sigs, pi_logits = split_mixture_params(params, output_dim, num_mixes)
    pis = softmax(pi_logits, t=temp)
    m = np.random.choice(range(len(pis)), p=pis)
    mus_vector = mus[m*output_dim:(m+1)*output_dim]
    sig_vector = sigs[m*output_dim:(m+1)*output_dim] * sigma_temp  # adjust for temperature
    cov_matrix = np.identity(output_dim) * sig_vector
    sample = np.random.multivariate_normal(mus_vector, cov_matrix, num_sample)
    return sample

def split_mixture_params(params, output_dim, num_mixes):
    mus = params[num_mixes:num_mixes*output_dim]
    sigs = params[num_mixes*output_dim:]
    pi_logits = params[:num_mixes]
    return mus, sigs, pi_logits

In [ ]:
from tensorflow.keras import regularizers


class MT_MDN_FT(Model):
    def __init__(self):
        super(MT_MDN, self).__init__()
        #shared layer  
        self.f1 = Dense(64,activation = 'linear')
        self.d1 = Dropout(0.01)
        self.f2 = Dense(100, kernel_regularizer=regularizers.l2(0.001), activation='relu')                                  
        self.d2 = Dropout(0.01)
        self.f3 = Dense(100, kernel_regularizer=regularizers.l2(0.001), activation='relu')
        self.f4 = Dense(100,kernel_regularizer=regularizers.l2(0.001), activation = 'relu')
        self.d3 = Dropout(0.01)
        self.f5 = Dense(100,kernel_regularizer=regularizers.l2(0.001),activation='relu')
        
        
        self.aux_f1 = Dense(32,kernel_regularizer=regularizers.l2(0.001),activation = 'linear')
        self.aux_f2 = Dense(64,activation = 'relu')
        
        
        #mdn网络前部分      
        self.fmixture = Dense(N_MIXES*(1+OUTPUT_DIMS+OUTPUT_DIMS*(OUTPUT_DIMS+1)//2),activation ='relu')
        
        
        # for TSS
        self.f21 = Dense(64, activation='relu')
        self.d21 = Dropout(0.01)
        self.f22 = Dense(100, kernel_regularizer=regularizers.l2(0.001),activation='relu')
        self.d22 = Dropout(0.01)
        self.f23 = Dense(128,kernel_regularizer=regularizers.l2(0.001),activation = 'relu')
        self.f24 = Dense(128,activation = 'relu')
        self.f25 = Dense(100,activation = 'relu')
        self.f26 = Dense(64,activation = 'relu')
        self.f27 = Dense(1,activation = 'linear',name = 'out2')
        
        # for chla
        self.f31 = Dense(100, activation='relu')
        self.d31 = Dropout(0.01)
        self.f32 = Dense(128, kernel_regularizer=regularizers.l2(0.001),activation='relu')
        self.d32 = Dropout(0.01)
        self.f33 = Dense(100,kernel_regularizer=regularizers.l2(0.001),activation = 'relu')
        self.d33 = Dropout(0.01)
        self.f34 = Dense(100,kernel_regularizer=regularizers.l2(0.001),activation = 'relu')
        self.d34 = Dropout(0.01)
        self.f35 = Dense(64,kernel_regularizer=regularizers.l2(0.001),activation = 'relu')
        self.d35 = Dropout(0.01)
        self.f36 = Dense(32,kernel_regularizer=regularizers.l2(0.001),activation = 'relu')
        self.fchla = Dense(1,activation = 'linear',name = 'out3')
        
        
        # for CDOM       
        self.f41 = Dense(128, activation='relu')
        self.d41 = Dropout(0.01)
        self.f42 = Dense(100,kernel_regularizer=regularizers.l2(0.001), activation='relu')
        self.d42 = Dropout(0.01)
        self.f43 = Dense(100,kernel_regularizer=regularizers.l2(0.001),activation = 'relu')
        self.f44 = Dense(64,kernel_regularizer=regularizers.l2(0.001),activation = 'relu')
        self.f45 = Dense(32,activation = 'relu')
        self.f46 = Dense(16,activation = 'relu')
        self.f47 = Dense(8,activation = 'relu')
        self.f48 = Dense(1,activation = 'linear',name = 'out4')

        
    def call(self, x):
        #shared layer
        x1 = x
        x2 = x[:,0:17]

        
        xtss1 = x1[:,12:14]
        xtss2 = x1[:,3:7]
        xtss = concatenate([xtss2,xtss1])
        xchla = x1[:,15:17]
        xchla2 = x1[:,3:7]
        xcdom = x1[:,0:15]  


        x2 = self.f1(x2)
        x2 = self.d1(x2)
        x2 = self.f2(x2)
        x2 = self.d2(x2)
        x2 = self.f3(x2)
        x2 = self.d3(x2)
        #x2 = self.f4(x2)
        x2 = self.f5(x2)
        
        
        #约束层
        mix_inputs = self.fmixture(x2)
        
        mdn_pi,mdn_mus,scale = tf.split(mix_inputs,
                            [N_MIXES,N_MIXES*OUTPUT_DIMS,N_MIXES*OUTPUT_DIMS*(OUTPUT_DIMS+1)//2] , axis=1)
        mdn_pi = tf.nn.softmax(mdn_pi, axis=-1) + tf.constant(1e-9)
        mdn_mus = tf.stack(tf.split(mdn_mus, N_MIXES,1),1)
        scale = tf.stack(tf.split(scale, N_MIXES, 1), 1) 
        scale = tfp.math.fill_triangular(scale, upper=False)
        norm  = tf.linalg.diag(tf.ones((1, 1, OUTPUT_DIMS)))
        sigma = tf.einsum('abij,abjk->abik', tf.transpose(scale, perm=[0,1,3,2]), scale)
        sigma+= epsilon * norm
        scale = tf.linalg.cholesky(sigma)
        #计算出的MDN模型的三个统计量
        out1 = concatenate([tf.reshape(mdn_pi, shape=[-1, N_MIXES]),
                            tf.reshape(mdn_mus,    shape=[-1, N_MIXES * OUTPUT_DIMS]),
                            tf.reshape(scale, shape=[-1, N_MIXES * OUTPUT_DIMS ** 2])],name = 'out1')    

    
    
        #    
        coefs = get_coefs(out1)
        outy = _get_top_estimate(coefs)
        
        ###################
        #下一层网络的输入
        outytss = tf.reshape(outy[:,0],shape = [-1,1])
        outychla = tf.reshape(outy[:,1],shape = [-1,1])
        outycdom = tf.reshape(outy[:,-1],shape = [-1,1])

        out01 = concatenate([xtss1,outytss])
        out02 = concatenate([xchla,outychla])
        out03 = concatenate([xcdom,outycdom])
        ##################
        
        
        # out2 TSS模型
        out2 = self.f21(out01)
        out2 = self.d21(out2)
        out2 = self.f22(out2)
        out2 = self.d22(out2)
        out2 = self.f23(out2)
        out2 = self.f24(out2)
        out2 = self.f25(out2)
        out2 = self.f26(out2)
        out2 = self.f27(out2)

        # out3  chla模型
        out3 = self.f31(out02)
        out3 = self.d31(out3)
        out3 = self.f32(out3)
        out3 = self.d32(out3)
        out3 = self.f33(out3)
        out3 = self.d33(out3)
        out3 = self.f34(out3)
        out3 = self.f36(out3)
        
        out3 = self.fchla(out3)
        
        
        # out4  cdom模型
        out4 = self.f41(out03)
        out4 = self.d41(out4)
        out4 = self.f42(out4)
#         out4 = self.d42(out4)
#         out4 = self.f43(out4)
#         out4 = self.f44(out4)
        out4 = self.f45(out4)
        out4 = self.f46(out4)
#         out4 = self.f47(out4)
        out4 = self.f48(out4)

        return out1, out2, out3, out4

In [ ]:
def huber(y1, y2):
    l = tf.keras.losses.Huber()
    huber_loss = l(y1, y2)
    return huber_loss
def lossmae(y1,y2):
    mae = tf.keras.losses.MeanAbsoluteError()
    maeloss = mae(y1,y2)
    return maeloss

# Mean Squared Logarithmic Error (MSLE)
def lossmsle(predict, label):
    loss = tf.reduce_mean(tf.losses.mean_squared_logarithmic_error(label, predict))
    return loss

# Root Mean Squared Error
def lossrmse(predict, label):
    loss = tf.sqrt(tf.reduce_mean(tf.losses.mean_squared_error(label, predict)))
    return loss


In [ ]:
model = MT_MDN()